<a href="https://colab.research.google.com/github/piesauce/oreilly-llm-course/blob/main/MCP_Server_oreilly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCP Server 101 (Simple): Weather Tools (Free API, No Key)

This notebook builds a **tiny MCP server** that exposes **2 tools** backed by the free **Open‑Meteo** endpoints:

- `search_locations(query)` → finds a city and returns lat/lon  
- `get_current_weather(latitude, longitude)` → current temperature + wind + weather code  
- `umbrella_advice(query)` → a friendly answer using precipitation probability (optional but fun)

✅ **No API keys.**  
✅ Runs in Colab.  
✅ Includes a minimal **stdio client test** (so you can demo MCP without any desktop app).

> **Important:** MCP stdio servers must not print to stdout. We use logging to stderr.


In [ ]:
!pip -q install -U "mcp[cli]" httpx nest_asyncio

import mcp
print("mcp version:", getattr(mcp, "__version__", "unknown"))

mcp version: unknown


In [ ]:
%%writefile weather_mcp_server.py
from typing import Any, Dict
import httpx
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("weather-mcp-simple")

GEOCODE_URL = "https://geocoding-api.open-meteo.com/v1/search"
FORECAST_URL = "https://api.open-meteo.com/v1/forecast"


@mcp.tool()
async def search_locations(query: str, count: int = 5) -> Dict[str, Any]:
    """Find a place (city) and return up to `count` matches with lat/lon."""
    params = {"name": query, "count": count, "language": "en", "format": "json"}
    async with httpx.AsyncClient(timeout=20) as client:
        r = await client.get(GEOCODE_URL, params=params)
        r.raise_for_status()
        data = r.json()

    results = data.get("results") or []
    return {
        "query": query,
        "results": [
            {
                "name": x.get("name"),
                "country": x.get("country"),
                "admin1": x.get("admin1"),
                "latitude": x.get("latitude"),
                "longitude": x.get("longitude"),
                "timezone": x.get("timezone"),
            }
            for x in results
        ],
    }


@mcp.tool()
async def get_current_weather(latitude: float, longitude: float, timezone: str = "auto") -> Dict[str, Any]:
    """Current weather at a lat/lon (temp, wind, weather_code)."""
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "timezone": timezone,
        "current": "temperature_2m,wind_speed_10m,wind_direction_10m,weather_code",
    }
    async with httpx.AsyncClient(timeout=20) as client:
        r = await client.get(FORECAST_URL, params=params)
        r.raise_for_status()
        data = r.json()

    current = data.get("current") or {}
    return {
        "timezone": data.get("timezone"),
        "latitude": latitude,
        "longitude": longitude,
        "current": {
            "time": current.get("time"),
            "temperature_2m": current.get("temperature_2m"),
            "wind_speed_10m": current.get("wind_speed_10m"),
            "wind_direction_10m": current.get("wind_direction_10m"),
            "weather_code": current.get("weather_code"),
        },
    }


@mcp.tool()
async def umbrella_advice(city: str) -> Dict[str, Any]:
    """Umbrella advice for the next 12 hours, based on precip probability."""
    geo = await search_locations(city, count=1)
    if not geo["results"]:
        return {"city": city, "advice": "Couldn't find that place. Try a bigger nearby city."}

    loc = geo["results"][0]
    lat, lon = loc["latitude"], loc["longitude"]

    params = {
        "latitude": lat,
        "longitude": lon,
        "timezone": "auto",
        "hourly": "precipitation_probability",
        "forecast_hours": 12,
    }
    async with httpx.AsyncClient(timeout=20) as client:
        r = await client.get(FORECAST_URL, params=params)
        r.raise_for_status()
        data = r.json()

    hourly = data.get("hourly") or {}
    probs = hourly.get("precipitation_probability") or []
    times = hourly.get("time") or []

    if not probs:
        return {"city": city, "location": loc, "advice": "No precipitation probability data available."}

    peak = max(probs)
    peak_i = probs.index(peak)
    peak_time = times[peak_i] if peak_i < len(times) else None

    if peak >= 60:
        advice = f"Bring an umbrella. Peak precip probability ~{peak}% around {peak_time}."
    elif peak >= 30:
        advice = f"Maybe bring an umbrella. Peak precip probability ~{peak}% around {peak_time}."
    else:
        advice = f"Probably no umbrella needed. Peak precip probability ~{peak}%."

    return {"city": city, "location": loc, "peak_probability": peak, "peak_time": peak_time, "advice": advice}


if __name__ == "__main__":
    # Default is stdio; explicit is fine too: mcp.run(transport="stdio")
    mcp.run()

Writing weather_mcp_server.py


In [ ]:
import json
import nest_asyncio
import sys
from pathlib import Path

nest_asyncio.apply()

from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client

server_params = StdioServerParameters(
    command=sys.executable,
    args=["weather_mcp_server.py"],
    env=None,
)

def unpack_call_result(result):
    """
    Robust parsing across MCP versions:
    - Prefer structuredContent if present
    - Else try to parse first TextContent as JSON
    - Else fall back to raw text
    """
    if hasattr(result, "structuredContent") and result.structuredContent:
        return result.structuredContent

    if getattr(result, "content", None):
        for c in result.content:
            if isinstance(c, types.TextContent):
                txt = c.text
                try:
                    return json.loads(txt)
                except Exception:
                    return {"text": txt}

    return {"text": str(result)}

async def run_test():
    err_path = Path("/tmp/mcp_server_stderr.log")
    with err_path.open("w") as errlog:
        async with stdio_client(server_params, errlog=errlog) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()

                tools = await session.list_tools()
                print("TOOLS:", [t.name for t in tools.tools])

                locs = await session.call_tool("search_locations", {"query": "Toronto", "count": 3})
                locs_data = unpack_call_result(locs)
                print("\nLOCATIONS:")
                print(json.dumps(locs_data, indent=2)[:1200])
                payload = locs_data.get("result", locs_data)   # <-- unwrap if needed
                first = (payload.get("results") or [None])[0]

                if not first:
                    print("No results; stopping.")
                    return

                wx = await session.call_tool(
                    "get_current_weather",
                    {"latitude": first["latitude"], "longitude": first["longitude"], "timezone": first.get("timezone") or "auto"},
                )
                wx_data = unpack_call_result(wx)
                print("\nCURRENT WEATHER:")
                print(json.dumps(wx_data, indent=2))

                umb = await session.call_tool("umbrella_advice", {"city": "Toronto"})
                umb_data = unpack_call_result(umb)
                print("\nUMBRELLA ADVICE:")
                print(json.dumps(umb_data, indent=2))

try:
    await run_test()
except BaseExceptionGroup as eg:
    print("\n--- MCP TEST FAILED (ExceptionGroup) ---")
    for i, e in enumerate(eg.exceptions, 1):
        print(f"[{i}] {type(e).__name__}: {e}")
    p = Path("/tmp/mcp_server_stderr.log")
    print("\n--- server stderr tail ---")
    print(p.read_text(errors="ignore")[-4000:] if p.exists() else "(no stderr file)")
except Exception as e:
    print("\n--- MCP TEST FAILED ---")
    print(type(e).__name__ + ":", e)
    p = Path("/tmp/mcp_server_stderr.log")
    print("\n--- server stderr tail ---")
    print(p.read_text(errors="ignore")[-4000:] if p.exists() else "(no stderr file)")

TOOLS: ['search_locations', 'get_current_weather', 'umbrella_advice']

LOCATIONS:
{
  "result": {
    "query": "Toronto",
    "results": [
      {
        "name": "Toronto",
        "country": "Canada",
        "admin1": "Ontario",
        "latitude": 43.70643,
        "longitude": -79.39864,
        "timezone": "America/Toronto"
      },
      {
        "name": "Toronto",
        "country": "United States",
        "admin1": "Ohio",
        "latitude": 40.46423,
        "longitude": -80.60091,
        "timezone": "America/New_York"
      },
      {
        "name": "Toronto",
        "country": "United States",
        "admin1": "Kansas",
        "latitude": 37.79893,
        "longitude": -95.94916,
        "timezone": "America/Chicago"
      }
    ]
  }
}

CURRENT WEATHER:
{
  "result": {
    "timezone": "America/Toronto",
    "latitude": 43.70643,
    "longitude": -79.39864,
    "current": {
      "time": "2026-02-24T22:30",
      "temperature_2m": -4.8,
      "wind_speed_10m": 10.1,

## 1) Write the MCP server (stdio)

## 2) Quick test: call the MCP tools from a tiny stdio client

## 3) (Optional) What to teach in a 1‑day intro

**Key concepts to emphasize:**
- What MCP is: *a standard way to expose tools/resources/prompts to LLM clients*
- Why stdio: simplest transport (works anywhere, easy to demo)
- Tools: typed functions (inputs → structured outputs)
- “Wrapper value”: validation, safety, normalized outputs, aggregation logic

**Suggested 60–90 min lab:**
1. Add a new tool: `get_daily_forecast(latitude, longitude, days=3)`
2. Make outputs more “LLM friendly”: add `summary` strings
3. Add simple input validation (lat/lon ranges, empty query)
